# DOTA 使用FCN做15类的异常检测, ResNet50对error_map分类

## 输出3*15通道 [x0,x1,...,x14], 查看wx 与  [x0,x1,...,x14]之间的距离

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import datetime
import time
import numpy as np

# 载入数据

In [2]:
#data_dir = r"D:/datasets/ShipVSField"
data_dir = r"D:\datasets\dota\Slices"
data_transform = {x:transforms.Compose([transforms.Resize([256,256]),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)])
                                      for x in ["train"] } #, "valid"

# 删除隐藏文件夹 .ipynb_checkpoints
for x in ["train", "valid"]:
    if os.path.exists( os.path.join(data_dir,x,".ipynb_checkpoints") ):
        print(u"Delete ".format(os.path.join(data_dir,x,".ipynb_checkpoints") ))
        os.removedirs(os.path.join(data_dir,x,".ipynb_checkpoints"))
    
image_datasets = {x:datasets.ImageFolder(root = os.path.join(data_dir,x),
                                        transform = data_transform[x]) for x in ["train"]} #, "valid"

dataloader = {x:torch.utils.data.DataLoader(dataset = image_datasets[x],
                                           batch_size = 8, #16,
                                           shuffle = True) for x in ["train"]}#, "valid"

In [3]:
X_example,y_example = next(iter( dataloader["train"] ) )
example_classes = image_datasets["train"].classes
index_classes = image_datasets["train"].class_to_idx
cls_num = len(index_classes)

print(index_classes)

{'baseball-diamond': 0, 'basketball-court': 1, 'bridge': 2, 'ground-track-field': 3, 'harbor': 4, 'helicopter': 5, 'large-vehicle': 6, 'plane': 7, 'roundabout': 8, 'ship': 9, 'small-vehicle': 10, 'soccer-ball-field': 11, 'storage-tank': 12, 'swimming-pool': 13, 'tennis-court': 14}


# 配置模型

In [5]:
# FCN_resnet50
num_classes = 3 * 15 # 15类
model_fcn = torchvision.models.segmentation.fcn_resnet50(pretrained=False, progress=True, num_classes=num_classes, aux_loss=None)

In [6]:
# Resnet50 分类
model_res = torchvision.models.resnet50(pretrained= False)
state_dict = torch.load("resnet50-19c8e357.pth")
model_res.load_state_dict(state_dict)

# 修改输入输出通道数
model_res.fc = torch.nn.Linear(2048, 15)
model_res.conv1 = torch.nn.Conv2d(3*15,64, kernel_size=7,padding=3,bias=False)

In [7]:
#加载模型参数
model_res.load_state_dict(torch.load('DOAT_Slices_FCN-ResNet50_model_last.pth')['model_res'])
model_fcn.load_state_dict(torch.load('DOAT_Slices_FCN-ResNet50_model_last.pth')['model_fcn'])

<All keys matched successfully>

In [12]:
y_fcn = model_fcn(X_example )['out'] # * masked_gaussian
print(y_fcn.shape)

cls_num = len(index_classes)

Classes = np.random.randint(low=0, high=cls_num, size=len(X_example))
y_class = torch.cat( [y_fcn[i:i+1,np.array(range(0,3))+3*Classes[i],:,:] for i in range(len(Classes))], axis=0)

torch.pow(X_example - y_class, 2).sum(), 

#torch.tensor(X_example.shape).prod(dim=0)

torch.Size([8, 45, 256, 256])


(tensor(216914.3750, grad_fn=<SumBackward0>),)

In [13]:
error_map = torch.pow(torch.cat([X_example]*cls_num, axis=1) - y_fcn, 2)
print(error_map.shape)
y_res = model_res(error_map)
y_res.shape

torch.Size([8, 45, 256, 256])


torch.Size([8, 15])

# 损失和优化函数

In [14]:
class SquaredErrorLoss(torch.nn.Module):
    def __init__(self):
        super(SquaredErrorLoss, self).__init__()
        
    def forward(self, X, Y, Classes):
        input_shape = torch.tensor(X.size())
        n = X.shape[1]
        index = torch.tensor((range(0,n))).to(Classes.device)
        Y_class = torch.cat( [Y[i:i+1,index+n*Classes[i],:,:] for i in range(len(Classes))], axis=0)
        Error = torch.pow(X - Y_class, 2).sum() / input_shape.prod()
        return Error
    
# FCN 损失和优化函数
loss_fcn = SquaredErrorLoss()

# ResNet损失和优化函数
loss_res = torch.nn.CrossEntropyLoss()

import itertools
optimizer = torch.optim.Adam(itertools.chain(model_fcn.parameters(),model_res.parameters()), lr=0.00001)

In [15]:
Classes = torch.tensor( np.random.randint(low=0, high=cls_num, size=len(X_example)) )
print(Classes)

loss_fcn(X_example,y_fcn, Classes)

tensor([ 6,  9, 11,  4,  2, 11,  9,  8], dtype=torch.int32)


tensor(0.0628, grad_fn=<DivBackward0>)

# 配置GPU，开始进行训练

In [16]:
import collections
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import time
import pdb

In [17]:
try:
    del X_example
    del y
    del y_class
    del error_map
except:
    1

In [16]:
torch.cuda.empty_cache()

In [19]:
# 配置GPU，开始进行训练
Use_gpu = torch.cuda.is_available()

if Use_gpu:
    print( torch.cuda.get_device_properties(device = 0) )
    model_fcn = model_fcn.cuda()
    model_res = model_res.cuda()
    
# 配置多GPU
if torch.cuda.device_count() > 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_fcn = torch.nn.DataParallel(model_fcn)
    model_res = torch.nn.DataParallel(model_res)
    
masked_gaussian = masked_gaussian.to(next(model_fcn.parameters()).device)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8192MB, multi_processor_count=20)
Let's use 2 GPUs!


In [20]:
epoch_n = 100
time_open = time.time()
f = open('Loss_log.txt', 'a+')
f.write("\nDOAT_Slices_FCN begin\n")

for epoch in range(epoch_n):
    print("Epoch {}/{}".format(epoch, epoch_n - 1))
    print("-"*10)
    
    for phase in ["train"]: # "valid"
        if phase == "train":
            print("Training...")
            model_fcn.train(True)
            model_res.train(True)
        else:
            print("Valid...")
            model_fcn.train(False)
            model_res.train(False) # 把BatchNormalization和DropOut固定住，不会取平均，而是用训练好的值
            
        running_loss = 0.0
        running_corrects = 0.0
        
        for batch, data in enumerate(dataloader[phase], start = 1): #组合为一个索引序列
            X, y = data
            if Use_gpu:
                X, y = Variable(X.cuda()), Variable(y.cuda())
            else:
                X, y = Variable(X), Variable(y)
            
            # 添加高斯掩膜
            y_fcn = model_fcn.forward(X * masked_gaussian.to(X.device))['out']
            
            error_map = torch.pow(torch.cat([X]*cls_num, axis=1) - y_fcn, 2)
            y_pred = model_res.forward(error_map)
            
            
            _, pred = torch.max(y_pred.data, 1)
            
            loss = loss_fcn(X, y_fcn, y) + loss_res(y_pred, y)
            
            # 对参数梯度的归零
            optimizer.zero_grad()
            if phase == "train":
                # 计算反向传播梯度值 P167
                loss.backward() 
                # 对节点的参数进行梯度更新 P169
                optimizer.step()
                
            running_loss += float(loss.data)
            running_corrects += float(torch.sum(pred == y.data))
            
            if batch%500 == 0 and phase == "train":
                print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}"
                      .format(batch, running_loss/batch, 100*running_corrects/batch/len(pred)))
                print("Current time {:.2f} min".format((time.time() - time_open)/60))
            
        epoch_loss = running_loss*16/len(image_datasets[phase])
        epoch_acc = 100*running_corrects/len(image_datasets[phase])
        
        print("{} Loss:{:.4f}  Acc:{:.4f}%".format(phase, epoch_loss, epoch_acc))
        time_end = time.time() - time_open
        print("Finish time {:.2f} min".format(time_end/60))
        
        if torch.cuda.device_count() > 1:
            torch.save({'model_res':model_res.module.state_dict(),
                        'model_fcn':model_fcn.module.state_dict()},
                        'DOAT_Slices_FCN-ResNet50_epoch_{}.pth'.format(epoch))
        else:
            torch.save({'model_res':model_res.state_dict(), 
                        'model_fcn':model_fcn.state_dict()},
                        'DOAT_Slices_FCN-ResNet50_epoch_{}.pth'.format(epoch))
    
        
# 清除显卡缓存
torch.cuda.empty_cache()
f.close()
if torch.cuda.device_count() > 1:
    torch.save({'model_res':model_res.module.state_dict(),
                'model_fcn':model_fcn.module.state_dict()},
                'DOAT_Slices_FCN-ResNet50_model_last.pth')
else:
    torch.save({'model_res':model_res.state_dict(), 
                'model_fcn':model_fcn.state_dict()},
                'DOAT_Slices_FCN-ResNet50_model_last.pth')

Epoch 0/99
----------
Training...


C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Batch 500, Train Loss:0.0466, Train ACC:99.3000
Current time 6.97 min
Batch 1000, Train Loss:0.0454, Train ACC:99.3000
Current time 13.87 min
Batch 1500, Train Loss:0.0450, Train ACC:99.4083
Current time 20.72 min


KeyboardInterrupt: 

In [1]:
phase

NameError: name 'phase' is not defined

In [28]:

if torch.cuda.device_count() > 1:
    torch.save({'model_res':model_res.module.state_dict(),
                'model_fcn':model_fcn.module.state_dict()},
                'DOAT_Slices_FCN-ResNet50_model_last.pth')
else:
    torch.save({'model_res':model_res.state_dict(), 
                'model_fcn':model_fcn.state_dict()},
                'DOAT_Slices_FCN-ResNet50_model_last.pth')

In [19]:
print("休眠",datetime.datetime.now())
os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")
os.system("rundll32.exe powrprof.dll,SetSuspendState")
print("休眠",datetime.datetime.now())

休眠 2022-07-26 17:52:10.801712
休眠 2022-07-27 11:04:16.536572


# Valid

In [9]:
# 配置GPU，开始进行训练
Use_gpu = torch.cuda.is_available()

if Use_gpu:
    print( torch.cuda.get_device_properties(device = 0) )
    model_fcn = model_fcn.cuda()
    model_res = model_res.cuda()
    
# 配置多GPU
if torch.cuda.device_count() > 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_fcn = torch.nn.DataParallel(model_fcn)
    model_res = torch.nn.DataParallel(model_res)
    
masked_gaussian = masked_gaussian.to(next(model_fcn.parameters()).device)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8192MB, multi_processor_count=20)
Let's use 2 GPUs!


In [17]:
time_open = time.time()
print("Valid...")

phase = "train"

model_fcn.train(False) # 把BatchNormalization和DropOut固定住，不会取平均，而是用训练好的值
model_res.train(False)         
    
running_loss = 0.0
running_corrects = 0.0
predict = [[] for i in range(len(example_classes))]

for batch, data in enumerate(dataloader[phase], start = 1): #组合为一个索引序列
    X, y = data
    if Use_gpu:
        X, y = Variable(X.cuda()), Variable(y.cuda())
    else:
        X, y = Variable(X), Variable(y)

    # 添加高斯掩膜
    y_fcn = model_fcn.forward(X * masked_gaussian.to(X.device))['out']

    error_map = torch.pow(torch.cat([X]*cls_num, axis=1) - y_fcn, 2)
    y_pred = model_res.forward(error_map)
    
    _, pred = torch.max(y_pred.data, 1)

    loss = loss_fcn(X, y_fcn, y) + loss_res(y_pred, y)
    
    # 记录预测结果
    for i in range(len(y)):
        predict[y[i]].append( y_pred[i].cpu().detach().numpy() )  #.argmax(dim=1)
    
    running_loss += float(loss.data)
    running_corrects += float(torch.sum(pred == y.data))

    if batch%500 == 0:# and phase == "train":
        print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}"
              .format(batch, running_loss/batch, 100*running_corrects/batch/len(pred)))
        print("Current time {:.2f} min".format((time.time() - time_open)/60))

epoch_loss = running_loss*16/len(image_datasets[phase])
epoch_acc = 100*running_corrects/len(image_datasets[phase])

print("{} Loss:{:.4f}  Acc:{:.4f}%".format(phase, epoch_loss, epoch_acc))
time_end = time.time() - time_open
print("Finish time {:.2f} min".format(time_end/60))

Valid...
Batch 500, Train Loss:0.0621, Train ACC:99.1500
Current time 2.85 min
Batch 1000, Train Loss:0.0625, Train ACC:99.2625
Current time 5.69 min
Batch 1500, Train Loss:0.0587, Train ACC:99.2750
Current time 8.50 min
Batch 2000, Train Loss:0.0583, Train ACC:99.2438
Current time 11.30 min
Batch 2500, Train Loss:0.0573, Train ACC:99.2550
Current time 14.06 min
Batch 3000, Train Loss:0.0565, Train ACC:99.2750
Current time 16.83 min
Batch 3500, Train Loss:0.0576, Train ACC:99.2393
Current time 19.58 min
Batch 4000, Train Loss:0.0570, Train ACC:99.2313
Current time 22.29 min
Batch 4500, Train Loss:0.0557, Train ACC:99.2500
Current time 25.01 min
Batch 5000, Train Loss:0.0553, Train ACC:99.2500
Current time 27.70 min
Batch 5500, Train Loss:0.0575, Train ACC:99.2477
Current time 30.40 min
Batch 6000, Train Loss:0.0570, Train ACC:99.2646
Current time 33.07 min
Batch 6500, Train Loss:0.0580, Train ACC:99.2558
Current time 35.73 min
Batch 7000, Train Loss:0.0579, Train ACC:99.2607
Current ti

In [21]:
pred_vec = [np.array([]) for i in range(len(example_classes))]
Correct,Num = 0,0
for cat in range(len(predict)):
    catname = list(index_classes.keys())[list(index_classes.values()).index(cat)]
    rate = np.sum(np.array([p.argmax() for p in predict[cat]]) == cat)/ len(predict[cat])
    print("{} : {}%, Num : {}".format(catname, rate*100, len(predict[cat])))
    
    Correct =  Correct + np.sum(np.array([p.argmax() for p in predict[cat]]) == cat)
    Num = Num + len(predict[cat])
    
    pred_vec[cat] = np.sum( predict[cat], axis=0 ) / len(predict[cat])

baseball-diamond : 94.4578313253012%, Num : 415
basketball-court : 96.31067961165049%, Num : 515
bridge : 99.62765957446808%, Num : 1880
ground-track-field : 98.15384615384616%, Num : 325
harbor : 98.54320160750167%, Num : 5972
helicopter : 96.66136724960255%, Num : 629
large-vehicle : 99.80765763058244%, Num : 16637
plane : 99.62311557788944%, Num : 7960
roundabout : 98.74371859296483%, Num : 398
ship : 99.48390450376861%, Num : 26933
small-vehicle : 99.13134400386069%, Num : 24866
soccer-ball-field : 86.29283489096574%, Num : 321
storage-tank : 98.50679741475373%, Num : 4487
swimming-pool : 99.71114962449451%, Num : 1731
tennis-court : 98.856416772554%, Num : 2361


In [25]:
import pandas as pd
pd.DataFrame(pred_vec).to_csv("pred_vec_FCN.csv")